In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ranker/ranker.json
/kaggle/input/aeroclub-recsys-2025/jsons_raw.tar.kaggle
/kaggle/input/aeroclub-recsys-2025/train.parquet
/kaggle/input/aeroclub-recsys-2025/sample_submission.parquet
/kaggle/input/aeroclub-recsys-2025/jsons_structure.md
/kaggle/input/aeroclub-recsys-2025/test.parquet


In [2]:
# Reading the training data 
import pyarrow.parquet as pq

train_data_file_name = "/kaggle/input/aeroclub-recsys-2025/train.parquet"
test_data_file_name = "/kaggle/input/aeroclub-recsys-2025/test.parquet"
n_rows = 5000000

parquet_file = pq.ParquetFile(train_data_file_name)
batch = next(parquet_file.iter_batches(batch_size=n_rows))
df_train = batch.to_pandas()

# df_train = pd.read_parquet(train_data_file_name, engine="fastparquet")

df_train.head(5)


,Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,...,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,selected
0,0,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T16:20:00,2024-06-15T15:40:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,370.0,16884.0,1
1,1,True,57323,123,S7/SU/UT,36,True,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,51125.0,0
2,2,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,53695.0,0
3,3,True,57323,123,S7/SU/UT,36,True,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,81880.0,0
4,4,True,57323,<NA>,S7/SU/UT,36,False,False,2024-06-15T14:50:00,2024-06-15T09:25:00,...,1.0,1,2087645,98ce0dabf6964640b63079fbafd42cbe,2024-05-17 03:03:08,TLKKJA/KJATLK,True,2240.0,86070.0,0


In [3]:
# Getting information about the dataframe
print(f"Train table:\n {df_train.describe()} \n{'=' * 10}")

Train table:
                  Id     companyID  corporateTariffCode  nationality  \
count  5.000000e+06  5.000000e+06            2178958.0    5000000.0   
mean   2.500000e+06  4.911230e+04           105.302213    35.713715   
min    0.000000e+00  1.663600e+04                  0.0          0.0   
25%    1.250000e+06  4.102200e+04                 75.0         36.0   
50%    2.500000e+06  5.416300e+04                103.0         36.0   
75%    3.749999e+06  6.006100e+04                152.0         36.0   
max    4.999999e+06  6.348200e+04                181.0         48.0   
std    1.443376e+06  1.205190e+04            45.734902     2.790215   

       legs0_segments0_baggageAllowance_quantity  \
count                               4.999571e+06   
mean                                2.670414e+00   
min                                 0.000000e+00   
25%                                 0.000000e+00   
50%                                 1.000000e+00   
75%                               

In [4]:
# Getting info about non-numerical data types
print(f"Non-numerical columns:\n {df_train.select_dtypes(exclude='number').describe()} \n{'=' * 10}")

Non-numerical columns:
                          requestDate
count                        5000000
mean   2024-06-21 03:28:36.638402816
min              2024-05-17 03:03:08
25%              2024-06-05 13:34:51
50%              2024-06-21 13:22:37
75%              2024-07-08 12:36:30
max              2024-07-19 14:55:18 


In [5]:
# Showing columns of train set
print(df_train.columns)
print(df_train.index)

Index(['Id', 'bySelf', 'companyID', 'corporateTariffCode', 'frequentFlyer',
       'nationality', 'isAccess3D', 'isVip', 'legs0_arrivalAt',
       'legs0_departureAt',
       ...
       'pricingInfo_isAccessTP', 'pricingInfo_passengerCount', 'profileId',
       'ranker_id', 'requestDate', 'searchRoute', 'sex', 'taxes', 'totalPrice',
       'selected'],
      dtype='object', length=126)
Index([      0,       1,       2,       3,       4,       5,       6,       7,
             8,       9,
       ...
       4999990, 4999991, 4999992, 4999993, 4999994, 4999995, 4999996, 4999997,
       4999998, 4999999],
      dtype='int64', length=5000000)


In [6]:
# Looking at specific "interesting" columns
cols_of_interest = [
    "Id",
    "profileId",
    "sex",
    "nationality",
    "isVip",
    "corporateTariffCode",
    "searchRoute",
    "selected",
    "pricingInfo_isAccessTP",
    "ranker_id"
]
print(f"Non-numerical columns:\n {df_train[cols_of_interest].describe()} \n{'=' * 10}")

Non-numerical columns:
                  Id     profileId  nationality  corporateTariffCode  \
count  5.000000e+06  5.000000e+06    5000000.0            2178958.0   
mean   2.500000e+06  2.541934e+06    35.713715           105.302213   
std    1.443376e+06  9.034005e+05     2.790215            45.734902   
min    0.000000e+00  8.130000e+02          0.0                  0.0   
25%    1.250000e+06  1.953616e+06         36.0                 75.0   
50%    2.500000e+06  2.863185e+06         36.0                103.0   
75%    3.749999e+06  3.301856e+06         36.0                152.0   
max    4.999999e+06  3.510039e+06         48.0                181.0   

           selected  pricingInfo_isAccessTP  
count  5.000000e+06            4.694111e+06  
mean   6.297400e-03            4.705847e-01  
std    7.910590e-02            4.991340e-01  
min    0.000000e+00            0.000000e+00  
25%    0.000000e+00            0.000000e+00  
50%    0.000000e+00            0.000000e+00  
75%    0.00000

In [7]:
import gc
del df_train
gc.collect()

0

In [8]:
# making validation, testing and training sets
# from sklearn.model_selection import train_test_split

# X, y = df_train.drop(["selected"], axis=1), df_train["selected"]

# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=0)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [9]:
# Getting the profile ids

users_train = pq.read_table('/kaggle/input/aeroclub-recsys-2025/train.parquet', columns=['profileId']).to_pandas()
users_test = pq.read_table('/kaggle/input/aeroclub-recsys-2025/test.parquet', columns=['profileId']).to_pandas()



In [10]:
# Counting the profile ids

unique_users_train_cnt = users_train['profileId'].value_counts()
print(unique_users_train_cnt)

unique_users_test_cnt = users_test['profileId'].value_counts()
print(unique_users_test_cnt)

profileId
1246816    28718
3335115    28500
3102963    27281
2814466    24950
2036978    24087
           ...  
3544224        1
770476         1
3515297        1
3077498        1
3350969        1
Name: count, Length: 32922, dtype: int64
profileId
1421682    26341
2432939    25308
2743931    23599
3301895    23061
3560801    21085
           ...  
3483781        1
2088684        1
3326287        1
3481946        1
3197756        1
Name: count, Length: 18981, dtype: int64


In [11]:
# Checking if all the test users were mentioned in the train dataset (No, there are new users in test subset)
print(f"Statement that all users in test dataset are mentioned in train dataset is "
        f"{set(users_test['profileId']).issubset(set(users_train['profileId']))}")

Statement that all users in test dataset are mentioned in train dataset is False


In [12]:
# Getting the profileIds of the users
unique_users_train = unique_users_train_cnt.index
unique_users_test = unique_users_test_cnt.index

In [13]:
# Getting the rankerIds 

rankers_train = pq.read_table('/kaggle/input/aeroclub-recsys-2025/train.parquet', columns=['ranker_id']).to_pandas()
rankers_test = pq.read_table('/kaggle/input/aeroclub-recsys-2025/test.parquet', columns=['ranker_id']).to_pandas()
unique_ranker_id_train = rankers_train['ranker_id'].unique()
unique_ranker_id_test = rankers_test['ranker_id'].unique()
unique_ranker_id_train_set = set(rankers_train['ranker_id'].unique())
unique_ranker_id_test_set = set(rankers_test['ranker_id'].unique())


From the analyzing the dataset above, the following idea of the learning is:

- Create several XGBoost classifiers models for any profile which is mentioned in the train dataset more than `min_freq` times. For everyone else, use the one united, trained on everyone XGBoost classifier

- XGBoost is most suitable since of the high dimensionality of the data provided with many uncontinuous features


In [14]:
'''
TODO LIST:
Read about the rankers (finish reading)
https://arrow.apache.org/docs/python/generated/pyarrow.parquet.ParquetFile.html#pyarrow.parquet.ParquetFile.read_row_groups
https://medium.com/predictly-on-tech/learning-to-rank-using-xgboost-83de0166229d
https://xgboost.readthedocs.io/en/stable/python/python_api.html#callback-api
'''

'\nTODO LIST:\nRead about the rankers (finish reading)\nhttps://arrow.apache.org/docs/python/generated/pyarrow.parquet.ParquetFile.html#pyarrow.parquet.ParquetFile.read_row_groups\nhttps://medium.com/predictly-on-tech/learning-to-rank-using-xgboost-83de0166229d\nhttps://xgboost.readthedocs.io/en/stable/python/python_api.html#callback-api\n'

In [15]:
def special_ranker_data(file_name, ranker_ids):
    '''
    Get the dataset for the specific rankers to train the model on (used since not enough RAM available to download everything at once)
    '''
    table = pq.read_table(
        file_name,
        filters=[('ranker_id', 'in', ranker_ids)]
    )
    return table.to_pandas()


In [16]:
"""
TODO: 
    Categorize the object values via scilitlearn
    Train the common ranker for everyone in the training dataset
    
TO READ:
    https://xgboost.readthedocs.io/en/stable/python/examples/learning_to_rank.html#sphx-glr-python-examples-learning-to-rank-py
    https://xgboost.readthedocs.io/en/stable/tutorials/learning_to_rank.html
"""

'\nTODO: \n    Categorize the object values via scilitlearn\n    Train the common ranker for everyone in the training dataset\n    \nTO READ:\n    https://xgboost.readthedocs.io/en/stable/python/examples/learning_to_rank.html#sphx-glr-python-examples-learning-to-rank-py\n    https://xgboost.readthedocs.io/en/stable/tutorials/learning_to_rank.html\n'

In [17]:
# Getting extra features for the dataframe
import datetime
from math import isnan

suffixes = ["_minute", "_hour", "_dotw", "_dotm", "_month"]
original_time_labels = [
            "legs0_departureAt_ordinal",
            "legs1_departureAt_ordinal",
            "legs0_arrivalAt_ordinal",
            "legs1_arrivalAt_ordinal",]
time_labels_precalc = [[time_label + suffix for suffix in suffixes] for time_label in original_time_labels]

def ordinal_time(dt):
    '''
    turns dt (datetime) into the ordinal representation. 
    Used to create another column
    '''
    if dt is None:
        return None

    # One sample from times: 2024-07-22T20:30:00
    date_format = "%Y-%m-%dT%H:%M:%S"
    return datetime.datetime.strptime(dt, date_format).toordinal()

def time_specification_columns(row, time_labels):
    """
    Create the hour, minute, day of the week, month and day of the month columns
        - time_labels - names of the columns with ordinal times
    """
    global time_labels_precalc

    suffixes = ["_minute", "_hour", "_dotw", "_dotm", "_month"]
    new_cols = pd.Series()
    
    for i, time_label in enumerate(time_labels):
        if row[time_label] is None or isnan(row[time_label]):
            for suffix in suffixes:
                new_cols[time_label + suffix] = None
        else:
            tt = datetime.datetime.timetuple(datetime.datetime.fromordinal(int(row[time_label])))
            new_cols[time_labels_precalc[i][0]] = tt.tm_min
            new_cols[time_labels_precalc[i][1]] = tt.tm_hour
            new_cols[time_labels_precalc[i][2]] = tt.tm_wday
            new_cols[time_labels_precalc[i][3]] = tt.tm_mday
            new_cols[time_labels_precalc[i][4]] = tt.tm_mon
            
    return new_cols
        

def overlay_length(row, arrival_time_label, departure_time_label):
    '''
    length of overlay
    arrival_time_label and departure_time_label are lables of the columns with
    the ordinal representations dates
    '''
    # checking that both entries are not None
    if row[arrival_time_label] and row[departure_time_label]:
        return departure_time - arrival_time
    return None
    
def count_number_of_layovers(row, airport_labels):
    """
    Counts number of layovers
        airport_labels - list of labels (with increasing segment order) of the compan
    """
    cnt = 0
    for a_label in arrival_time_labels:
        cnt += 1
    return cnt

def is_overnight(row, departure_time_label, arrival_time_label):
    """
    Checks if the flight is overnight.
        - departure_time_label - label of the column with ordinal departure time
        - arrival_time_label - label of the column with ordinal arrival time
    """

    if isnan(row[arrival_time_label]) or isnan(row[arrival_time_label]):
        return None
    
    dep_dt = datetime.datetime.fromordinal(int(row[departure_time_label]))
    if dep_dt.hour >= 20 or dep_dt.hour <= 10:
        return True

    arr_dt = datetime.datetime.fromordinal(int(row[arrival_time_label]))
    if arr_dt.hour >= 22 or arr_dt <= 12:
        return True

    return False
    

def total_flight_length(row, departure_time_label, arrival_time_label):
    """
    Calculates the total time of the flight. 
        - departure_time_label - label of the time of the initial departure (ordinal)
        - arrival_time_label - label of the time of the final arrival (ordinal)
    """
    
    return row[arrival_time_label] - row[departure_time_label]
    
    
def number_of_unique_entries(row, labels):
    """
    Count the number of unique entries
        - labels - list of the labels of unique entries 
    """
    been = set()
    been.add(None)

    for label in labels:
        been.add(row[label])

    been.remove(None)
    return len(been)
    

def extra_features(df):
    """
    Adding the features (columns) to the dataframe of the flights
        - df - pandas dataframe
    """
    global original_time_labels
    # adding ordinal time columns
    df["legs0_departureAt_ordinal"] = df["legs0_departureAt"].apply(ordinal_time).astype('float')
    df["legs0_arrivalAt_ordinal"] = df["legs0_arrivalAt"].apply(ordinal_time).astype('float')
    df["legs1_departureAt_ordinal"] = df["legs1_departureAt"].apply(ordinal_time).astype('float')
    df["legs1_arrivalAt_ordinal"] = df["legs1_arrivalAt"].apply(ordinal_time).astype('float')

    # adding ordinal duration time
    df["legs0_duration"] = df.apply(total_flight_length, args=("legs0_departureAt_ordinal", "legs0_arrivalAt_ordinal"), axis=1)
    df["legs1_duration"] = df.apply(total_flight_length, args=("legs1_departureAt_ordinal", "legs1_arrivalAt_ordinal"), axis=1)

    # Adding number of different companies 
    legs0_companies = [
        "legs0_segments0_marketingCarrier_code",
        "legs0_segments1_marketingCarrier_code",
        "legs0_segments2_marketingCarrier_code",
        "legs0_segments3_marketingCarrier_code",
    ]
    legs1_companies = [
        "legs1_segments0_marketingCarrier_code",
        "legs1_segments1_marketingCarrier_code",
        "legs1_segments2_marketingCarrier_code",
        "legs1_segments3_marketingCarrier_code",
    ]
    df["legs0_different_companies_cnt"] = df.apply(number_of_unique_entries, args=(legs0_companies,), axis=1)
    df["legs1_different_companies_cnt"] = df.apply(number_of_unique_entries, args=(legs1_companies,), axis=1)

    # Counnting number of layovers
    legs0_arrival_ariports_iata = []
    legs1_arrival_ariports_iata = []
    for i in range(4):
        legs0_arrival_ariports_iata.append(f"legs0_segments{i}_arrivalTo_airport_iata")
        legs1_arrival_ariports_iata.append(f"legs1_segments{i}_arrivalTo_airport_iata")

    df["legs0_layovers_num"] = df.apply(number_of_unique_entries, args=(legs0_arrival_ariports_iata,), axis=1)
    df["legs1_layovers_num"] = df.apply(number_of_unique_entries, args=(legs1_arrival_ariports_iata,), axis=1)

    # Checking if the flighs are overnight
    df["legs0_is_overnight"] = df.apply(is_overnight, args=("legs0_departureAt_ordinal", "legs0_arrivalAt_ordinal"), axis=1).astype('float')
    df["legs1_is_overnight"] = df.apply(is_overnight, args=("legs1_departureAt_ordinal", "legs1_arrivalAt_ordinal"), axis=1).astype('float')

    # adding the time columns (TAKES TOO MUCH TIME TO PROCESS)
    # print("getting time cols...")
    # time_cols = df.apply(
    #     time_specification_columns,
    #     args=(original_time_labels,), 
    #     axis=1
    # ).astype('float')
    # df[time_cols.columns] = time_cols
    # print("Done")
    
    return df

In [18]:
# creating the model
import xgboost as xgb


ranker = xgb.XGBRanker(
    objective='rank:ndcg',
    eval_metric='ndcg',
    n_estimators=200,
    device="cuda",
    enable_categorical=True,
)

In [19]:
# Training the model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import ndcg_score
import random

def train_over_dataset(step_size):
    global ranker
    # train the ranker over entire dataset
    rankers_num = len(unique_ranker_id_train)
    iter_num = rankers_num // step_size * step_size
    
    for i in range(0, iter_num, step_size):
        
        # getting the data
        ranker_ids = set(unique_ranker_id_train[i:i + step_size:])
        
        rank_df = special_ranker_data(train_data_file_name, ranker_ids)
        print("read data")
        # sorting by ranker_id feature
        rank_df = rank_df.sort_values(by=['ranker_id', 'selected'], ascending=[True, False])
        
        
        X, y = rank_df.drop(["selected", "ranker_id"], axis=1), rank_df["selected"]
        X = extra_features(X)
    
        qid = np.array(LabelEncoder().fit_transform(rank_df['ranker_id'])) + i
        # print(qid)
        print(f"{i} - {i + step_size - 1} loaded")
    
        # Split the data in the train, val and test sets
        # train_qids, else_qids = train_test_split(np.unique(qid), test_size=0.3, random_state=0)
        # val_qids, test_qids = train_test_split(else_qids, test_size=0.5, random_state=0)
        train_qids, val_qids = train_test_split(np.unique(qid), test_size=0.2, random_state=0)
        
        train_indices = np.isin(qid, train_qids)
        val_indices = np.isin(qid, val_qids)
        # test_indices = np.isin(qid, test_qids)
    
        # Transforming all not accesible data types into the category data type
        accessible_dtypes = ['int', 'float', 'bool', 'category']
        for col in X.columns:
            # print(col, X[col].dtype)
            # print(X.iloc[0][col])
            if X[col].dtype not in accessible_dtypes:
                X[col] = X[col].astype('category')
        
        X_train, y_train, qid_train = X[train_indices], y[train_indices], qid[train_indices]
        X_val, y_val, qid_val = X[val_indices], y[val_indices], qid[val_indices]
        # X_test, y_test, qid_test = X[test_indices], y[test_indices], qid[test_indices]
        es = xgb.callback.EarlyStopping(
            rounds=10,
            save_best=True,
            maximize=True,
            metric_name="ndcg",
        )
        ranker.set_params(callbacks=[es])
        
        # fitting the data
        if i != 0:
            ranker = ranker.fit(
                X=X_train,
                y=y_train,
                qid=qid_train,
                eval_set=[(X_val, y_val)],
                eval_qid=[qid_val],
                xgb_model=ranker,
            )
        else:
            ranker = ranker.fit(
                X=X_train,
                y=y_train,
                qid=qid_train,
                eval_set=[(X_val, y_val)],
                eval_qid=[qid_val],
            )
    
        # printing the best validation ndcg
        res = ranker.evals_result()['validation_0']['ndcg']
        print(f"There are {len(res)} estimators tried with maximum validation {max(res)}")
    

    

In [22]:
output_model_path = "/kaggle/working/ranker.json"
input_model_path = "/kaggle/input/ranker/ranker.json"

In [20]:
# train the ranker
import random
import os

# train only if the models don't exist
if not os.path.exists(input_model_path):
    random.seed(0)
    n_trains = 5
    for i in range(n_trains):
        step_size=random.randint(1000, 2000)
        print(f"Iteration {i}, step_size = {step_size}")
        train_over_dataset(step_size)

Iteration 0, step_size = 8155
read data
0 - 8154 loaded
There are 11 estimators tried with maximum validation 0.8747203683794738
read data
8155 - 16309 loaded
There are 11 estimators tried with maximum validation 0.8218128297791759
read data
16310 - 24464 loaded
There are 11 estimators tried with maximum validation 0.7857042721867253
read data


KeyboardInterrupt: 

In [ ]:
# saving the ranker
if not os.path.exists(input_model_path):
    ranker.save_model(output_model_path)
    ranker.save_model(input_model_path)

In [23]:
# loading the ranker
ranker.load_model(input_model_path)

In [20]:
# looking at example submission
sample_submission = pq.read_table('/kaggle/input/aeroclub-recsys-2025/sample_submission.parquet').to_pandas()
sample_submission 

,Id,ranker_id,selected
18144679,18144679,c9373e5f772e43d593dd6ad2fa90f67a,178
18144680,18144680,c9373e5f772e43d593dd6ad2fa90f67a,363
18144681,18144681,c9373e5f772e43d593dd6ad2fa90f67a,277
18144682,18144682,c9373e5f772e43d593dd6ad2fa90f67a,183
18144683,18144683,c9373e5f772e43d593dd6ad2fa90f67a,55
...,...,...,...
25043143,25043143,c5622e0de0594bde95a4dd8c1fcff7ac,9
25043144,25043144,c5622e0de0594bde95a4dd8c1fcff7ac,2
25043145,25043145,c5622e0de0594bde95a4dd8c1fcff7ac,1
25043146,25043146,c5622e0de0594bde95a4dd8c1fcff7ac,11


In [43]:
# prediciton
prediction = pq.read_table('/kaggle/input/aeroclub-recsys-2025/test.parquet', columns=['Id', 'ranker_id']).to_pandas()
prediction.index = prediction['Id']
prediction.index.name = None
prediction['selected'] = np.full(len(prediction), -1)
prediction

,Id,ranker_id,selected
18144679,18144679,c9373e5f772e43d593dd6ad2fa90f67a,-1
18144680,18144680,c9373e5f772e43d593dd6ad2fa90f67a,-1
18144681,18144681,c9373e5f772e43d593dd6ad2fa90f67a,-1
18144682,18144682,c9373e5f772e43d593dd6ad2fa90f67a,-1
18144683,18144683,c9373e5f772e43d593dd6ad2fa90f67a,-1
...,...,...,...
25043143,25043143,c5622e0de0594bde95a4dd8c1fcff7ac,-1
25043144,25043144,c5622e0de0594bde95a4dd8c1fcff7ac,-1
25043145,25043145,c5622e0de0594bde95a4dd8c1fcff7ac,-1
25043146,25043146,c5622e0de0594bde95a4dd8c1fcff7ac,-1


In [51]:
# prediciton
step_size = 2000
rankers_num = len(unique_ranker_id_test)

print(rankers_num)

for i in range(0, rankers_num, step_size):

    
    # getting the data
    j = min(i + step_size, rankers_num)
    ranker_ids = set(unique_ranker_id_test[i:j])
    
    rank_df = special_ranker_data(test_data_file_name, ranker_ids)
    print("read data")

    # getting only specific values
    for ranker_id in ranker_ids:
        indices = rank_df['ranker_id'] == ranker_id
        X = rank_df[indices]
        X = X.drop(["ranker_id"], axis=1)
        X = extra_features(X)
        accessible_dtypes = ['int', 'float', 'bool', 'category']
        for col in X.columns:
            if X[col].dtype not in accessible_dtypes:
                X[col] = X[col].astype('category')
        
        y_pred = ranker.predict(X)
        # getting the order
        y_pred = np.argsort(y_pred)
        y_pred = len(y_pred) - y_pred 

        row_is = np.arange(len(y_pred))
        idxs = X.index[row_is]
        prediction.loc[idxs, 'selected'] = y_pred

    print(f"{i} through {i + step_size - 1} completed")

45231
read data
0 through 1999 completed
read data
2000 through 3999 completed
read data
4000 through 5999 completed
read data
6000 through 7999 completed
read data
8000 through 9999 completed
read data
10000 through 11999 completed
read data
12000 through 13999 completed
read data
14000 through 15999 completed
read data


KeyboardInterrupt: 

In [ ]:
print(prediction)

In [ ]:
prediction.to_csv('/kaggle/working/submission.csv')